In [ ]:
langchain==0.1.13
streamlit==1.32.2
chromadb==0.4.24
pypdf==4.1.0
langchain-community
huggingface-hub
sentence-transformers

In [1]:
!pip install -r a.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.0 MB/s eta 0:00:0

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ricepaper/vi-gemma-2b-RAG")
model = AutoModelForCausalLM.from_pretrained("ricepaper/vi-gemma-2b-RAG")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [1]:
from langchain.chains import RetrievalQA
from langchain.callbacks.manager import CallbackManager
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import os
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceEmbeddings

# --- Configuration ---
HUGGINGFACEHUB_API_TOKEN = "XXXXXXXXXXXXXXXXXXXXXXX"  # Replace with your actual token
PDF_FILE_PATH = "/content/The_Lightning_Thief_-_Percy_Jackson_1-10.pdf"  # Replace with the path to your PDF file
QUESTION = "What is the main topic of this document?"  # Replace with your question
PERSIST_DIRECTORY = "jj"
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
MODEL_KWARGS = {'device': 'cpu'}
ENCODE_KWARGS = {'normalize_embeddings': False}
REPO_ID = "ricepaper/vi-gemma-2b-RAG"
TEMPERATURE = 0.9

# --- Setup ---
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

# Create directories if they don't exist
if not os.path.exists('files'):
    os.mkdir('files')

if not os.path.exists(PERSIST_DIRECTORY):
    os.mkdir(PERSIST_DIRECTORY)

# --- Embeddings ---
embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    model_kwargs=MODEL_KWARGS,
    encode_kwargs=ENCODE_KWARGS
)

# --- Prompt Template ---
template = """You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative. Try to give answer in tabular and shortcut.

    Context: {context}
    History: {history}

    User: {question}
    Chatbot:"""

prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

# --- Memory ---
memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
    input_key="question"
)

# --- Vector Store ---
if not os.path.exists(os.path.join(PERSIST_DIRECTORY, "index")):
    # Load and process the PDF only if the index doesn't exist
    loader = PyPDFLoader(PDF_FILE_PATH)
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=0,
        length_function=len
    )
    all_splits = text_splitter.split_documents(data)

    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embeddings,
        persist_directory=PERSIST_DIRECTORY
    )
    vectorstore.persist()
else:
    vectorstore = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings
    )

# --- Language Model ---
llm = HuggingFaceEndpoint(
    repo_id=REPO_ID,
    temperature=TEMPERATURE,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

# --- Retriever ---
retriever = vectorstore.as_retriever()

# --- Retrieval QA Chain ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": memory,
    }
)

# --- Run the QA Chain ---
response = qa_chain(QUESTION)

# --- Print the Result ---
print(f"Question: {QUESTION}")
print(f"Answer: {response['result']}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative. Try to give answer in tabular and shortcut.

    Context: I could start at any point in my short miserable life 
to prove it, but things really started going bad last May, 
when our sixth-grade class took a field trip to Manhattan— 
twenty-eight mental-case kids and two teachers on a yellow 
school bus, heading to the Metropolitan Museum of Art to 
look at ancient Greek and Roman stuff. 
I know—it sounds like torture. Most Yancy field trips 
were. 
But Mr. Brunner, our Latin teacher, was leading this 
trip, so I had hopes. 
Mr. Brunner was this middle-aged guy in a motorized 
wheelchair. He had thinning hair and a scruffy beard and a 
frayed tweed jacket, which always smelled like coffee. You 
wouldn't think he'd be cool, 

ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='api-inference.huggingface.co', port=443): Read timed out. (read timeout=120)"), '(Request ID: c111a6bd-0614-4371-9688-a58c1cd26f88)')

In [ ]:
!pip install requests
import requests
from langchain.chains import RetrievalQA
from langchain.callbacks.manager import CallbackManager
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import os
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceEmbeddings

# --- Configuration ---
HUGGINGFACEHUB_API_TOKEN = "XXXXXXXXXXXXXXXXXXXXXXX"  # Replace with your actual token
PDF_FILE_PATH = "/content/The_Lightning_Thief_-_Percy_Jackson_1-10.pdf"  # Replace with the path to your PDF file
QUESTION = "What is the main topic of this document?"  # Replace with your question
PERSIST_DIRECTORY = "jj"
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
MODEL_KWARGS = {'device': 'cpu'}
ENCODE_KWARGS = {'normalize_embeddings': False}
REPO_ID = "ricepaper/vi-gemma-2b-RAG"
TEMPERATURE = 0.9

# --- Setup ---
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

# Create directories if they don't exist
if not os.path.exists('files'):
    os.mkdir('files')

if not os.path.exists(PERSIST_DIRECTORY):
    os.mkdir(PERSIST_DIRECTORY)

# --- Embeddings ---
embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    model_kwargs=MODEL_KWARGS,
    encode_kwargs=ENCODE_KWARGS
)

# --- Prompt Template ---
template = """You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative. Try to give answer in tabular and shortcut.

    Context: {context}
    History: {history}

    User: {question}
    Chatbot:"""

prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

# --- Memory ---
memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
    input_key="question"
)

# --- Vector Store ---
if not os.path.exists(os.path.join(PERSIST_DIRECTORY, "index")):
    # Load and process the PDF only if the index doesn't exist
    loader = PyPDFLoader(PDF_FILE_PATH)
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=0,
        length_function=len
    )
    all_splits = text_splitter.split_documents(data)

    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embeddings,
        persist_directory=PERSIST_DIRECTORY
    )
    vectorstore.persist()
else:
    vectorstore = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings
    )

# --- Language Model ---
llm = HuggingFaceEndpoint(
    repo_id=REPO_ID,
    temperature=TEMPERATURE,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
    #  Increase the timeout for the Hugging Face API call
    request_timeout=300
)

# --- Retriever ---
retriever = vectorstore.as_retriever()

# --- Retrieval QA Chain ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": memory,
    }
)

# --- Run the QA Chain ---
response = qa_chain(QUESTION)

# --- Print the Result ---
print(f"Question: {QUESTION}")
print(f"Answer: {response['result']}")

In [2]:
from langchain.chains import RetrievalQA
from langchain.callbacks.manager import CallbackManager
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import os
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceEmbeddings
import requests
# --- Configuration ---
HUGGINGFACEHUB_API_TOKEN = "XXXXXXXXXXXXXXXXXXXXXXX"  # Replace with your actual token
PDF_FILE_PATH = "/content/The_Lightning_Thief_-_Percy_Jackson_1-10.pdf"  # Replace with the path to your PDF file
QUESTION = "What is the main topic of this document?"  # Replace with your question
PERSIST_DIRECTORY = "jj"
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
MODEL_KWARGS = {'device': 'cpu'}
ENCODE_KWARGS = {'normalize_embeddings': False}
REPO_ID = "ricepaper/vi-gemma-2b-RAG"
TEMPERATURE = 0.9

# --- Setup ---
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

# Create directories if they don't exist
if not os.path.exists('files'):
    os.mkdir('files')

if not os.path.exists(PERSIST_DIRECTORY):
    os.mkdir(PERSIST_DIRECTORY)

# --- Embeddings ---
embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    model_kwargs=MODEL_KWARGS,
    encode_kwargs=ENCODE_KWARGS
)

# --- Prompt Template ---
template = """You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative. Try to give answer in tabular and shortcut.

    Context: {context}
    History: {history}

    User: {question}
    Chatbot:"""

prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

# --- Memory ---
memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
    input_key="question"
)

# --- Vector Store ---
if not os.path.exists(os.path.join(PERSIST_DIRECTORY, "index")):
    # Load and process the PDF only if the index doesn't exist
    loader = PyPDFLoader(PDF_FILE_PATH)
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=0,
        length_function=len
    )
    all_splits = text_splitter.split_documents(data)

    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embeddings,
        persist_directory=PERSIST_DIRECTORY
    )
    vectorstore.persist()
else:
    vectorstore = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings
    )

# --- Language Model ---
llm = HuggingFaceEndpoint(
    repo_id=REPO_ID,
    temperature=TEMPERATURE,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
    request_timeout=300
)

# --- Retriever ---
retriever = vectorstore.as_retriever()

# --- Retrieval QA Chain ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": memory,
    }
)

# --- Run the QA Chain ---
response = qa_chain(QUESTION)

# --- Print the Result ---
print(f"Question: {QUESTION}")
print(f"Answer: {response['result']}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no long



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative. Try to give answer in tabular and shortcut.

    Context: I could start at any point in my short miserable life 
to prove it, but things really started going bad last May, 
when our sixth-grade class took a field trip to Manhattan— 
twenty-eight mental-case kids and two teachers on a yellow 
school bus, heading to the Metropolitan Museum of Art to 
look at ancient Greek and Roman stuff. 
I know—it sounds like torture. Most Yancy field trips 
were. 
But Mr. Brunner, our Latin teacher, was leading this 
trip, so I had hopes. 
Mr. Brunner was this middle-aged guy in a motorized 
wheelchair. He had thinning hair and a scruffy beard and a 
frayed tweed jacket, which always smelled like coffee. You 
wouldn't think he'd be cool, 

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [ ]:
from langchain.chains import RetrievalQA
from langchain.callbacks.manager import CallbackManager
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import os
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceEmbeddings

# --- Configuration ---
HUGGINGFACEHUB_API_TOKEN = "XXXXXXXXXXXXXXXXXXXXXXX"  # Replace with your actual token
PDF_FILE_PATH = "/content/PDF-RAG/Clas Blomberg - Physics of life-Elsevier Science (2007).pdf"  # Replace with the path to your PDF file
QUESTION = "What is the main topic of this document?"  # Replace with your question
PERSIST_DIRECTORY = "jj"
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
MODEL_KWARGS = {'device': 'cpu'}
ENCODE_KWARGS = {'normalize_embeddings': False}
REPO_ID = "mistralai/Mistral-7B-Instruct-v0.2"
TEMPERATURE = 0.9

# --- Setup ---
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

# Create directories if they don't exist
if not os.path.exists('files'):
    os.mkdir('files')

if not os.path.exists(PERSIST_DIRECTORY):
    os.mkdir(PERSIST_DIRECTORY)

# --- Embeddings ---
embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    model_kwargs=MODEL_KWARGS,
    encode_kwargs=ENCODE_KWARGS
)

# --- Prompt Template ---
template = """You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative. Try to give answer in tabular and shortcut.

    Context: {context}
    History: {history}

    User: {question}
    Chatbot:"""

prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

# --- Memory ---
memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
    input_key="question"
)

# --- Vector Store ---
if not os.path.exists(os.path.join(PERSIST_DIRECTORY, "index")):
    # Load and process the PDF only if the index doesn't exist
    loader = PyPDFLoader(PDF_FILE_PATH)
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=0,
        length_function=len
    )
    all_splits = text_splitter.split_documents(data)

    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embeddings,
        persist_directory=PERSIST_DIRECTORY
    )
    vectorstore.persist()
else:
    vectorstore = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings
    )

# --- Language Model ---
llm = HuggingFaceEndpoint(
    repo_id=REPO_ID,
    temperature=TEMPERATURE,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

# --- Retriever ---
retriever = vectorstore.as_retriever()

# --- Retrieval QA Chain ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": memory,
    }
)

# --- Run the QA Chain ---
response = qa_chain(QUESTION)

# --- Print the Result ---
print(f"Question: {QUESTION}")
print(f"Answer: {response['result']}")